In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210403.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,30/03/2021 18:30:19,Bad Lieutenant: Port of Call New Orleans (2009),Moby Dick (1956),Big Deal on Madonna Street (1958),Bad Santa (2003),IT'S FRENCH,[Default to Nicolas Cage],BIG MONKEY,I REALLY HATE THESE GUYS,PREDICTING THE INTERNET FOR FUN AND PROFIT
1,30/03/2021 18:30:22,Bad Lieutenant: Port of Call New Orleans (2009),Moby Dick (1956),Big Deal on Madonna Street (1958),Bad Santa (2003),[Default to Nicolas Cage],I REALLY HATE THESE GUYS,BIG MONKEY,PREDICTING THE INTERNET FOR FUN AND PROFIT,IT'S FRENCH
2,30/03/2021 18:32:49,Big Deal on Madonna Street (1958),Moby Dick (1956),Bad Lieutenant: Port of Call New Orleans (2009),Bad Santa (2003),BIG MONKEY,PREDICTING THE INTERNET FOR FUN AND PROFIT,IT'S FRENCH,I REALLY HATE THESE GUYS,NaN
3,30/03/2021 18:33:26,Bad Lieutenant: Port of Call New Orleans (2009),Moby Dick (1956),Big Deal on Madonna Street (1958),Bad Santa (2003),I REALLY HATE THESE GUYS,PREDICTING THE INTERNET FOR FUN AND PROFIT,[Default to Nicolas Cage],BIG MONKEY,IT'S FRENCH
4,30/03/2021 22:14:23,Moby Dick (1956),Big Deal on Madonna Street (1958),Bad Santa (2003),Bad Lieutenant: Port of Call New Orleans (2009),I REALLY HATE THESE GUYS,BIG MONKEY,IT'S FRENCH,PREDICTING THE INTERNET FOR FUN AND PROFIT,[Default to Nicolas Cage]
5,31/03/2021 19:26:27,Bad Santa (2003),Bad Lieutenant: Port of Call New Orleans (2009),Big Deal on Madonna Street (1958),Moby Dick (1956),IT'S FRENCH,BIG MONKEY,I REALLY HATE THESE GUYS,PREDICTING THE INTERNET FOR FUN AND PROFIT,[Default to Nicolas Cage]
6,31/03/2021 21:29:44,Bad Lieutenant: Port of Call New Orleans (2009),Big Deal on Madonna Street (1958),Moby Dick (1956),Bad Santa (2003),BIG MONKEY,IT'S FRENCH,PREDICTING THE INTERNET FOR FUN AND PROFIT,[Default to Nicolas Cage],I REALLY HATE THESE GUYS
7,01/04/2021 17:25:04,Big Deal on Madonna Street (1958),Moby Dick (1956),NaN,NaN,BIG MONKEY,IT'S FRENCH,I REALLY HATE THESE GUYS,[Default to Nicolas Cage],PREDICTING THE INTERNET FOR FUN AND PROFIT


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Bad Lieutenant: Port of Call New Orleans (2009)',
       'Moby Dick (1956)', 'Big Deal on Madonna Street (1958)',
       'Bad Santa (2003)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Bad Lieutenant: Port of Call New Orleans (2009)': <Candidate('Bad Lieutenant: Port of Call New Orleans (2009)')>,
 'Moby Dick (1956)': <Candidate('Moby Dick (1956)')>,
 'Big Deal on Madonna Street (1958)': <Candidate('Big Deal on Madonna Street (1958)')>,
 'Bad Santa (2003)': <Candidate('Bad Santa (2003)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Bad Lieutenant: Port of Call New Orleans (2009), Moby Dick (1956), Big Deal on Madonna Street (1958), Bad Santa (2003))>,
 <Ballot(Bad Lieutenant: Port of Call New Orleans (2009), Moby Dick (1956), Big Deal on Madonna Street (1958), Bad Santa (2003))>,
 <Ballot(Big Deal on Madonna Street (1958), Moby Dick (1956), Bad Lieutenant: Port of Call New Orleans (2009), Bad Santa (2003))>,
 <Ballot(Bad Lieutenant: Port of Call New Orleans (2009), Moby Dick (1956), Big Deal on Madonna Street (1958), Bad Santa (2003))>,
 <Ballot(Moby Dick (1956), Big Deal on Madonna Street (1958), Bad Santa (2003), Bad Lieutenant: Port of Call New Orleans (2009))>,
 <Ballot(Bad Santa (2003), Bad Lieutenant: Port of Call New Orleans (2009), Big Deal on Madonna Street (1958), Moby Dick (1956))>,
 <Ballot(Bad Lieutenant: Port of Call New Orleans (2009), Big Deal on Madonna Street (1958), Moby Dick (1956), Bad Santa (2003))>,
 <Ballot(Big Deal on Madonna Street (1958), Moby Dick (1956))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                                          Votes  Status
-----------------------------------------------  -------  --------
Bad Lieutenant: Port of Call New Orleans (2009)        4  Elected
Big Deal on Madonna Street (1958)                      2  Rejected
Moby Dick (1956)                                       1  Rejected
Bad Santa (2003)                                       1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                     Votes  Status
------------------------------------------  -------  --------
BIG MONKEY                                        3  Hopeful
IT'S FRENCH                                       2  Hopeful
I REALLY HATE THESE GUYS                          2  Hopeful
[Default to Nicolas Cage]                         1  Rejected
PREDICTING THE INTERNET FOR FUN AND PROFIT        0  Rejected

ROUND 2
Candidate                                     Votes  Status
------------------------------------------  -------  --------
BIG MONKEY                                        3  Hopeful
I REALLY HATE THESE GUYS                          3  Hopeful
IT'S FRENCH                                       2  Rejected
[Default to Nicolas Cage]                         0  Rejected
PREDICTING THE INTERNET FOR FUN AND PROFIT        0  Rejected

FINAL RESULT
Candidate                                     Votes  Status
------------------------------------------  ------